# **Assignment 4: some database operations**
- **you will learn:** using joins, aggregation, groupby in Pandas
- **task:**  See section 4.2 below
- **deadline:** 17.11.2025
- [Pandas documentation](https://pandas.pydata.org/docs/index.html)
- 📝 **Reminder:** Sync your GitHub repository with the main course repository, update your project in PyCharm, and after completing the assignment, commit and push your changes back to GitHub.

In [ ]:
import numpy as np
import pandas as pd

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)

In [ ]:
# -------------------------------
# 1) Build synthetic datasets
# -------------------------------
# sensor readouts (long-format)
np.random.seed(42)
N_SENSORS = 6
sensors = [f'S{i+1}' for i in range(N_SENSORS)]

start = pd.Timestamp('2025-11-01 00:00')
periods = 24 * 6 # six measurements per hour for 24 hours
freq = '10min' # 10-minute frequency
idx = pd.date_range(start=start, periods=periods, freq=freq)

rows = []
for sensor in sensors:
    base_temp = 15 + 5 * np.random.rand() # baseline temp different per sensor
    base_hum = 40 + 20 * np.random.rand()
    noise_t = np.random.normal(0, 1.5, size=len(idx))
    noise_h = np.random.normal(0, 5.0, size=len(idx))
    # add a diurnal temperature variation
    hours = (idx.hour + idx.minute/60.0)
    diurnal = 6 * np.sin(2 * np.pi * (hours - 6) / 24)
    
    temps = base_temp + diurnal + noise_t
    hums = np.clip(base_hum - 0.5 * diurnal + noise_h, 0, 100)
    
    for ts, t, h in zip(idx, temps, hums):
        rows.append({'sensor_id': sensor, 'timestamp': ts, 'temperature': t, 'humidity': h})

sensor_df = pd.DataFrame(rows)

# add some missing values intentionally and some outliers
sensor_df.loc[sensor_df.sample(frac=0.01, random_state=1).index, 'temperature'] = np.nan
sensor_df.loc[sensor_df.sample(frac=0.005, random_state=2).index, 'humidity'] = np.nan
# inject outliers
outlier_idx = sensor_df.sample(frac=0.002, random_state=3).index
sensor_df.loc[outlier_idx, 'temperature'] += np.random.choice([20, -15], size=len(outlier_idx))

print('--- sensor_df (head) ---')
print(sensor_df.head())

# sensor metadata (wide)
meta = pd.DataFrame({
'sensor_id': sensors,
'location': ['Room A', 'Room B', 'Room C', 'Room A', 'Room D', 'Room B'],
'type': ['thermo-hygro', 'thermo-hygro', 'thermo', 'thermo-hygro', 'hygro', 'thermo']
})

# events table (sparse)
events = pd.DataFrame([
{'sensor_id': 'S1', 'timestamp': start + pd.Timedelta(hours=5), 'event': 'maintenance'},
{'sensor_id': 'S3', 'timestamp': start + pd.Timedelta(hours=8, minutes=20), 'event': 'calibration'},
{'sensor_id': 'S2', 'timestamp': start + pd.Timedelta(hours=18), 'event': 'spike_detected'}
])

In [ ]:
# -------------------------------
# 2) MultiIndex: set index as (sensor_id, timestamp)
# -------------------------------
sensor_mi = sensor_df.set_index(['sensor_id', 'timestamp']).sort_index()
print('\n--- sensor_mi (index sample) ---')
print(sensor_mi.head())

# show selecting using MultiIndex
print('\nSelect all readings for S2 between two times:')
print(sensor_mi.loc['S2'].between_time('06:00', '09:00').head())

In [ ]:
# -------------------------------
# 3) Multi-level columns example: create aggregated summary with MultiIndex columns
# -------------------------------
agg_funcs = {
'temperature': ['mean', 'std', 'min', 'max'],
'humidity': ['mean', 'std']
}
summary_by_sensor = sensor_df.groupby('sensor_id').agg(agg_funcs)
# flatten/unflatten demonstration: keep as MultiIndex columns
print('\n--- summary_by_sensor (MultiIndex columns) ---')
print(summary_by_sensor.head())

# rename columns to readable MultiIndex (level names)
summary_by_sensor.columns.names = ['measurement', 'stat']
print('\nColumns levels:', summary_by_sensor.columns.names)

# access a specific subtable using xs
print('\nTemperature means:')
print(summary_by_sensor.xs('temperature', axis=1)[['mean', 'std']])

In [ ]:
summary_by_sensor.columns = [
    f"{measurement}_{stat}"
    for measurement, stat in summary_by_sensor.columns
]

print('\n--- summary_by_sensor (flat columns) ---')
print(summary_by_sensor.head())

In [ ]:
# -------------------------------
# 4) Advanced groupby with named aggregation and custom functions
# -------------------------------
# We'll compute per-sensor and per-location aggregations using named aggregations
sensor_with_meta = sensor_df.merge(meta, on='sensor_id', how='left')

agg_named = sensor_with_meta.groupby(['location']).agg(
temp_mean=('temperature', 'mean'),
temp_med=('temperature', 'median'),
temp_iqr=('temperature', lambda x: np.subtract(*np.percentile(x.dropna(), [75, 25]))),
humidity_mean=('humidity', 'mean'),
n_readings=('temperature', 'count')
)
print('\n--- agg_named by location ---')
print(agg_named)

# custom aggregation using apply (slower but expressive) -> compute time-of-day sensitivity

def day_night_diff(group):
    group = group.copy()
    # daytime mean vs nighttime mean
    group['hour'] = group['timestamp'].dt.hour
    daytime = group.loc[(group['hour'] >= 6) & (group['hour'] < 18), 'temperature']
    nighttime = group.loc[(group['hour'] < 6) | (group['hour'] >= 18), 'temperature']
    return pd.Series({'day_minus_night': daytime.mean() - nighttime.mean()})

sens_daynight = sensor_df.merge(meta, on='sensor_id').groupby('sensor_id').apply(day_night_diff, include_groups=False)
print('\n--- day_minus_night per sensor ---')
print(sens_daynight.head())

In [ ]:
# Note on merge vs join
# pandas.merge() is the core function for all merges/joins.
# df.merge() and df.join() are convenient wrappers:
# - df.join() defaults to left join on df1's index and df2's index.
# - df.merge() defaults to inner join on columns, but can join on indexes.
# Use merge() for flexibility; join() can save typing in simple index-based left joins.

In [ ]:
# -------------------------------
# 5) pivot_table & crosstab
# -------------------------------
# pivot_table: average temperature per sensor per hour of day
sensor_df['hour'] = sensor_df['timestamp'].dt.hour
pv = pd.pivot_table(sensor_df, values='temperature', index='sensor_id', columns='hour', aggfunc='mean')
print('\n--- pivot_table mean temperature by sensor x hour ---')
print(pv.iloc[:, :6]) # show first 6 hours as sample

# crosstab: count of readings per sensor per type
sensor_meta_joined = sensor_df.merge(meta, on='sensor_id')
ct = pd.crosstab(sensor_meta_joined['sensor_id'], sensor_meta_joined['type'])
print('\n--- crosstab counts sensor x type ---')
print(ct)

In [ ]:
# -------------------------------
# 6) Merging and joining examples (all possibilities)
# -------------------------------
# inner merge (only sensors present in both)
m_inner = sensor_df.merge(meta, on='sensor_id', how='inner')
# left merge (keeps all readings)
m_left = sensor_df.merge(meta, on='sensor_id', how='left')
# right and outer
m_right = sensor_df.merge(meta, on='sensor_id', how='right')
m_outer = sensor_df.merge(meta, on='sensor_id', how='outer')

print('\nmerge sizes: inner={}, left={}, right={}, outer={}'.format(
len(m_inner), len(m_left), len(m_right), len(m_outer)
))

# join on index: prepare two DataFrames with indexes
df_a = sensor_df.sample(50, random_state=4).set_index(['sensor_id', 'timestamp']).sort_index()
df_b = pd.DataFrame({'battery': np.random.randint(20, 100, size=30)},
index=df_a.index[:30])
# left join using index
joined = df_a.join(df_b, how='left')

print('\n--- joined (index join) sample ---')
print(joined.head())

# concat: stack vertically and horizontally
concat_v = pd.concat([sensor_df.head(5), sensor_df.tail(5)], axis=0)
concat_h = pd.concat([sensor_df.head(5).reset_index(drop=True).iloc[:, :3],
sensor_df.head(5).reset_index(drop=True).iloc[:, 3:]], axis=1)

# combine_first: combine two sources preferring left's non-null values
left = sensor_df.head(10).copy()
right = left.copy()
right.loc[right.sample(frac=0.3, random_state=5).index, 'temperature'] = np.nan
combined = right.combine_first(left)
print('\n--- combine_first example ---')
print(combined.head())

In [ ]:
# -------------------------------
# 7) Reshaping: stack/unstack and melt
# -------------------------------
# create a small pivot and show stack/unstack
small = sensor_df[sensor_df['sensor_id'].isin(['S1', 'S2'])].head(12)
pv_small = small.pivot(index='timestamp', columns='sensor_id', values='temperature')
print('\n--- pv_small pivot (timestamp x sensor) ---')
print(pv_small.head())
print('\nstacked -> unstacked roundtrip (demonstration)')
stacked = pv_small.stack()
print(stacked.head())
print(stacked.unstack().head())

# melt: convert wide back to long
melted = pv_small.reset_index().melt(id_vars='timestamp', var_name='sensor_id', value_name='temperature')
print('\n--- melted back to long ---')
print(melted.head())

In [ ]:
# -------------------------------
# 8) Query: expressive filtering
# -------------------------------
# Use DataFrame.query for readable boolean expressions
q = sensor_df.query('temperature > 25 and humidity < 50')
print(f'\nNumber of readings with temperature>25 and humidity<50: {len(q)}')

# boolean indexing vs query for more complex expressions
expr = "(temperature > temperature.mean()) & (humidity < humidity.quantile(0.25))"
# careful: can't use temperature.mean() inside query easily; compute first
temp_mean = sensor_df['temperature'].mean()
hum_q25 = sensor_df['humidity'].quantile(0.25)
q2 = sensor_df.query('@temp_mean < temperature and humidity < @hum_q25')
print(f'\nUsing external variables in query, matches: {len(q2)}')

In [ ]:
# -------------------------------
# 9) Aggregations with groupby + transform and filter
# -------------------------------
# find sensors whose median temperature is > global median
global_median = sensor_df['temperature'].median()
sensor_medians = sensor_df.groupby('sensor_id')['temperature'].median()
hot_sensors = sensor_medians[sensor_medians > global_median].index.tolist()
print('\nSensors with median temperature > global median:', hot_sensors)

# use transform to broadcast group statistic back to rows
sensor_df['sensor_temp_median'] = sensor_df.groupby('sensor_id')['temperature'].transform('median')
# filter rows where temperature is above group's median
above_group_median = sensor_df[sensor_df['temperature'] > sensor_df['sensor_temp_median']]
print('\nRows above their sensor median (sample):')
print(above_group_median.head())

# groupby.filter to keep only sensors with at least 200 readings
sensors_with_enough = sensor_df.groupby('sensor_id').filter(lambda g: len(g) >= 200)
print('\nSensors kept (len>=200):', sensors_with_enough['sensor_id'].unique())

In [ ]:
# -------------------------------
# 10) MultiIndex columns: produce aggregated table with hierarchical columns and plot
# -------------------------------
agg_multi = sensor_df.groupby('sensor_id').agg(
temp_mean=('temperature', 'mean'),
temp_std=('temperature', 'std'),
hum_mean=('humidity', 'mean'),
hum_std=('humidity', 'std')
)
# create MultiIndex columns manually
agg_multi.columns = pd.MultiIndex.from_tuples([('temperature', 'mean'), ('temperature', 'std'), ('humidity', 'mean'), ('humidity', 'std')])
print('\n--- agg_multi (MultiIndex columns) ---')
print(agg_multi)

In [ ]:
# -------------------------------
# 11) Rolling, expanding and time-based resampling (for sensor S2)
# -------------------------------
s2 = sensor_df[sensor_df['sensor_id'] == 'S2'].set_index('timestamp').sort_index()
# rolling 30-minute window (freq is 10min so 3 observations -> use time-based window)
s2['temp_roll_30min'] = s2['temperature'].rolling('30min').mean()
# humidity diff next reading -> shift(-1)
s2['humidity_next_diff'] = s2['humidity'].shift(-1) - s2['humidity']
# boolean: temp increased >5C and humidity dropped >20 compared to previous measurement
s2['temp_prev_diff'] = s2['temperature'] - s2['temperature'].shift(1)
s2['hum_prev_diff'] = s2['humidity'].shift(1) - s2['humidity']
s2['extreme_change'] = (s2['temp_prev_diff'] > 5) & (s2['hum_prev_diff'] > 20)

## Task Specification: Aggregations, Joins, GroupBy, and Rolling Analysis with Pandas

---

### **Task Overview**
This assignment simulates a **customer purchase analytics workflow** using Pandas. You will work with timestamped transaction data and customer metadata, performing:

- Group-based aggregations
- Time-based analysis
- Rolling window calculations within groups
- Joining customer metadata from another dataset

---

### **Datasets**
You are provided with two datasets:

| File | Description |
|------|-------------|
| `data/purchases.csv` | Timestamped purchase data (contains `Customer_id`, `Timestamp`, `Price`, etc.) |
| `data/customer_info.csv` | Customer metadata (contains `Customer_id`, `Born_date`, `City`, etc.) |

---

### **1. Load and Prepare the Data**
- **Task 1.1:** Load the two CSV files into Pandas DataFrames:
  - Use `df` for `purchases.csv`
  - Use `customer_info` for `customer_info.csv`

- **Task 1.2:** Convert the `Timestamp` column in `df` to Pandas `datetime` format.

- **Task 1.3:** Set `Timestamp` as the **index** of `df` and **sort** the index in ascending order.

---

### **2. Join Customer Metadata**
- **Task 2.1:** Merge the purchase DataFrame (`df`) with the customer metadata DataFrame (`customer_info`) on the `Customer_id` column.

- **Task 2.2:** The resulting DataFrame should include the columns `Born_date` and `City` from the `customer_info` DataFrame.

---

### **3. GroupBy and Aggregations**
Using the merged DataFrame:

- **Task 3.1:** Group by **Customer_id** and compute:
  - Total revenue (`sum(Price)`)
  - Number of purchases (`count`)
  - Average purchase price (`mean(Price)`)

- **Task 3.2:** Create a new feature by extracting the **month name** (e.g., `"January"`, `"February"`) from the `Timestamp`.

- **Task 3.3:** Group by **City** and **Month** and compute:
  - Total monthly revenue per city (`sum(Price)`)

---

### **4. Rolling Window Analysis (Per Customer)**
Perform rolling window calculations **within each customer group**:

- Inside each customer group:
  - **Sort** the DataFrame by `Timestamp`.

- Compute the following:
  - A **3-purchase rolling average** of `Price`.
  - A `diff()` column showing the change in price compared to the previous purchase.

> **Note:** The rolling operations must be computed **within each customer group**, not across all customers.

---

### **5. Advanced Filtering**
- Do the same as in 4 only for customers from **Berlin**, make sure only necessary values are joined (that is filter first, join second).

- Do the same as in 4 only for rows where the `Price` of the current purchase is greater than **20%** than the previous purchase (`Price`), make sure only necessary values are joined (that is filter first, join second).

---

### **6. Custom Lambda Function for GroupBy Operations**
- Apply a **custom lambda function** within a `groupby()` operation to filter or modify the data. For example:
  - Use a lambda to **filter** customers who have purchased more than a specific amount within a given timeframe (e.g., greater than $1000 in total purchases during the last quarter).
  - Apply custom transformations to grouped data using the lambda function.

---

### 7. Time-Based Analysis
- Perform additional time-based analysis:
  - Calculate **monthly growth rate** for total revenue per customer (i.e., compare total revenue of the current month with the previous month).
  - Create a **cumulative sum** of purchases over time per customer.
